In [1]:
import os
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn import tree
import numpy as np
import pandas as pd
import random
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

In [2]:
max_T = 500
thresh = 1e-4 # Hyper-
#set random seed to make results reproducible
seed = 1234
np.random.seed(seed)
random.seed(seed)

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [4]:
experiment_info = {}
experiment_info["K"] = []
experiment_info["budget"] = []
experiment_info["Error"] = []
experiment_info["Equal opportunity difference"] = []

# Read Dataset

In [5]:
path = "../predict-responsibly/data/"
dataset = "recidivism"
protected_feature = "sex"

In [7]:
for K in [1,2,3,4,5]:
    db = np.load(os.path.join(path,"compas","{}{}_{}.npz".format(dataset,K,protected_feature)))
    x_val = db["x_train"]
    y_val = db["y_train"]
    x_test = db["x_test"]
    y_test = db["y_test"]
    logits = np.log(db["ydm_train"]/(1-db["ydm_train"]))
    test_logits = np.log(db["ydm_test"]/(1-db["ydm_test"]))
    # print("x_val",x_val.shape)
    # print("y_val",y_val.shape)
    # print("x_test",x_test.shape)
    # print("y_test",y_test.shape)
    # print("logits",logits.shape)
    # print("test_logits",test_logits.shape)
    # get the validation index

    valid_index = np.arange(len(x_val))
    for budget in [1,0.1]:
        best_epoch, best_acc = -1,0
        best_logits_valid = logits
        predicted_test_logits = test_logits
        #use budget % of the validation set for training
        valid_index =random.sample(list(valid_index), int(len(valid_index) * budget))


        # get the idxs1 and idxs2
        idxs1 = random.sample(list(valid_index), int(len(valid_index) * 0.7))
        idxs2 = list(set(valid_index) - set(idxs1))

        # save the coefficients
        # h_models = []
        # losses = []
        for t in range(max_T):
            # probs_heldout = sess_run(tf.nn.sigmoid(logits), x_val[idxs2], latent_val[idxs2], sess)
            # logits updates every iteration, thus the probs_heldout is different
            probs_heldout = sigmoid(logits)[idxs2]
            #heldout_loss = np.mean(-y_val[idxs2] * np.log(probs_heldout + 1e-20) - (1-y_val[idxs2]) * np.log(1-probs_heldout + 1e-20))
            heldout_acc =  np.mean((probs_heldout>0.5)==y_val[idxs2])
            # probs = sess_run(tf.nn.sigmoid(logits), x_val, latent_val ,sess)
            # 应为logits是更新的，所以probs也是更新的
            probs = sigmoid(logits)
            val_loss = np.mean(-y_val * np.log(probs + 1e-20) - (1 - y_val) * np.log(1 - probs + 1e-20))
            val_acc = np.mean((probs > 0.5) == y_val)
            # losses.append(val_loss)
            if heldout_acc > best_acc:
                # print("Update!")
                best_epoch = t
                best_acc = heldout_acc
                best_logits_valid = logits
                predicted_test_logits = test_logits
            # delta = res(probs,y_val)
            residual = probs - y_val
            for i in range(3):
                # define the control
                control_idx1 = db["ydm_train"][idxs1]>0.5
                control_idx2 = db["ydm_train"][idxs2]>0.5
                if i==0: 
                    # X_1, f1(x) > 1/2
                    temp_s = control_idx1
                    temp_s_heldout = control_idx2
                elif i ==1:
                    # X_0, f0(x) ≤ 1/2
                    temp_s = 1- control_idx1
                    temp_s_heldout = 1- control_idx2
                else:
                    # X
                    temp_s = np.ones_like(control_idx1)
                    temp_s_heldout = np.ones_like(control_idx2)
                # get the fresh sample for training
                samples1 = np.where(temp_s == 1)[0]
                samples2 = np.where(temp_s_heldout == 1)[0]
                # train the regression model 
                clf = Ridge(alpha=1) # 如果我不变呢
                # clf = DecisionTreeRegressor(max_depth = 5)
                # 如果要把protected feature 去掉 可以考率生成一个新的x_train、valid
                # 我没有去拟合他的 res 那个function，我直接拟合已知的residual （f(x) - y）
                clf.fit(x_val[idxs1][samples1],residual[idxs1][samples1])
                clf_prediction = clf.predict(x_val[idxs2][samples2])
                corr = np.mean(clf_prediction * residual[idxs2][samples2])
                # print(t, i, corr)
                if corr > thresh:
                    # h_models.append(clf)
                    # h = (tf.matmul(latent_ph, tf.constant(np.expand_dims(clf.coef_,-1),
                    #                                         dtype=tf.float32))[:,0] + clf.intercept_)
                    #here, we update h
                    h = clf.predict(x_val)
                    h_test = clf.predict(x_test)
                    # print(h.shape)
                    # print(h)
                    # when update the logits we only update the logits of current set
                    # logits -= .1 * h * s
                    control = db["ydm_train"]>0.5
                    #here ydm_test is the f_0(x_test) 
                    control_test = db["ydm_test"]>0.5
                    if i == 0:
                        # update logits of X_1
                        s = control
                        s_test = control_test
                    elif i == 1:
                        # update logits of X_0
                        s = 1 - control
                        s_test = 1 - control_test
                    else:
                        # update all logits (This might never used)
                        s = np.ones_like(control)
                        s_test = np.ones_like(control_test)
                    logits -= .1 * h * s
                    # update the test logits accordingly
                    test_logits -= .1 * h_test * s_test
                    break
            # 如果 i == 2 说明没有在X_0,X_1找到合适的h，那么就不更新了
            if i==2:
                break
        #Huangrui add
        EPS = 1e-8
        Y_hat = sigmoid(predicted_test_logits)>0.5
        A = db["attr_test"]
        Y = db["y_test"]
        # print("A",A.shape)
        # print("Y",Y.shape)
        TP = np.multiply(Y, Y_hat)
        mask0 = np.multiply(Y,1-A)
        mask1 = np.multiply(Y,A)
        TP0 = np.multiply(TP, mask0)
        TP1 = np.multiply(TP, mask1)
        tpr0 = np.sum(TP0) / (np.sum(mask0) + EPS)
        tpr1 = np.sum(TP1) / (np.sum(mask1) + EPS)
        #get the error rate of the test set
        error = np.mean(Y_hat != Y)
        eod = np.abs(tpr0 - tpr1)
        print("K={}, budget={}, error={}, eod={}".format(K,budget,error,eod))
        experiment_info["K"].append(K)
        experiment_info["budget"].append(budget)
        experiment_info["Error"].append(error)
        experiment_info["Equal opportunity difference"].append(eod)

K=1, budget=1, error=0.2793991416309013, eod=0.03109298140733674
K=1, budget=0.1, error=0.3034334763948498, eod=0.06621224940278447
K=2, budget=1, error=0.2657793044224989, eod=0.08655400821251147
K=2, budget=0.1, error=0.26449119793902964, eod=0.08448729982063041
K=3, budget=1, error=0.38686131386861317, eod=0.03543999257516206
K=3, budget=0.1, error=0.3911550021468441, eod=0.023341352804922555
K=4, budget=1, error=0.29841133533705455, eod=0.002836494075710738
K=4, budget=0.1, error=0.3027050236152855, eod=0.006618486141736435
K=5, budget=1, error=0.23711340206185566, eod=0.08426590534354961
K=5, budget=0.1, error=0.23324742268041238, eod=0.07039967369602806


In [8]:
import pandas as pd
df = pd.DataFrame.from_dict(experiment_info)
#save df to csv
df.to_csv("results/recidivism_sex.csv",index=False)

In [11]:
# Now get the error and eod of human
eods = []
errors = []
for K in [1,2,3,4,5]:
    db = np.load(os.path.join(path,"compas","{}{}_{}.npz".format(dataset,K,protected_feature)))
    Y_hat = db["ydm_test"]>0.5
    A = db["attr_test"]
    Y = db["y_test"]
    # print("A",A.shape)
    # print("Y",Y.shape)
    TP = np.multiply(Y, Y_hat)
    mask0 = np.multiply(Y,1-A)
    mask1 = np.multiply(Y,A)
    TP0 = np.multiply(TP, mask0)
    TP1 = np.multiply(TP, mask1)
    tpr0 = np.sum(TP0) / (np.sum(mask0) + EPS)
    tpr1 = np.sum(TP1) / (np.sum(mask1) + EPS)
    #get the error rate of the test set
    error = np.mean(Y_hat != Y)
    eod = np.abs(tpr0 - tpr1)
    errors.append(error)
    eods.append(eod)
    print("K={}, error={}, eod={}".format(K,error,eod))

K=1, error=0.26824034334763946, eod=0.027215449276761228
K=2, error=0.24903392013739803, eod=0.07588144784135853
K=3, error=0.3276084156290253, eod=0.06727892412711123
K=4, error=0.2589094031773293, eod=0.044623159236297116
K=5, error=0.23797250859106528, eod=0.06703099506657462


In [12]:
np.mean(errors)

0.2683529181764915

In [13]:
np.mean(eods)

0.05640599510962054